<a href="https://colab.research.google.com/github/gracelin0711/20250218/blob/main/EX04_03_%E5%AE%A2%E6%88%B6%E5%88%86%E7%BE%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義

將 性別 欄位 Label Encoding
將客戶分成 3 群，並解讀各群特質
計算 k = 2 ~ 15 的 Calinski-Harbasz Score，找出最佳 k 值

## 資料收集

In [ ]:
!wget -O car_models.csv https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv


--2025-05-27 07:56:15--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2839 (2.8K) [text/plain]
Saving to: ‘car_models.csv’

car_models.csv      100%[===================>]   2.77K  --.-KB/s    in 0s      

2025-05-27 07:56:15 (30.4 MB/s) - ‘car_models.csv’ saved [2839/2839]



In [ ]:
import pandas as pd # 引入 pandas 函式庫

url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'

# 將下載的 CSV 檔案讀取到一個 pandas DataFrame 中
df = pd.read_csv(url)
df.head()

,性別,年齡,收入（千）,消費指數（1~100）
0,女,74,38,81
1,女,51,71,91
2,女,30,65,10
3,女,88,49,17
4,女,55,48,70


## 資料前處理

### 資料清理

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   性別           200 non-null    object
 1   年齡           200 non-null    int64 
 2   收入（千）        200 non-null    int64 
 3   消費指數（1~100）  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.4+ KB


### 探索性分析

In [ ]:
df_cor = df.drop(columns=['性別']).corr()
df_cor

,年齡,收入（千）,消費指數（1~100）
年齡,1.000000,0.031519,-0.127454
收入（千）,0.031519,1.000000,0.031476
消費指數（1~100）,-0.127454,0.031476,1.000000


### 資料分割

### 類別轉換

In [ ]:
df['性別'].unique()

array(['女', '男'], dtype=object)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
# 修正 categories 的形狀，使其符合單一特徵的需求，將所有類別放在一個內層列表中
encoder = OrdinalEncoder(categories=[['女', '男']])
df[['性別']] = encoder.fit_transform(df[['性別']])
df

,性別,年齡,收入（千）,消費指數（1~100）
0,0.0,74,38,81
1,0.0,51,71,91
2,0.0,30,65,10
3,0.0,88,49,17
4,0.0,55,48,70
...,...,...,...,...
195,1.0,86,84,82
196,1.0,59,52,30
197,0.0,63,29,61
198,1.0,67,80,9


### 特徵縮放

## 模型訓練

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

KMeans(n_clusters=3)

In [ ]:
# 為每筆資料加上分群結果
df['cluster'] = kmeans.labels_
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,0.0,74,38,81,1
1,0.0,51,71,91,2
2,0.0,30,65,10,0
3,0.0,88,49,17,0
4,0.0,55,48,70,1
...,...,...,...,...,...
195,1.0,86,84,82,2
196,1.0,59,52,30,0
197,0.0,63,29,61,1
198,1.0,67,80,9,0


## 模型評估

In [ ]:
df.groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.589744,57.141026,62.166667,20.435897
1,0.610169,52.542373,77.457627,78.525424
2,0.507937,47.698413,37.238095,68.015873


In [ ]:
# 評估分群效果
from sklearn.metrics import calinski_harabasz_score
score = calinski_harabasz_score(df.drop(columns='cluster'), kmeans.labels_)


## 模型調整

In [ ]:
df_nocluster = df.drop(columns='cluster')
for i in range(2, 16):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(df_nocluster)
  score = calinski_harabasz_score(df_nocluster, kmeans.labels_)
  print(f'k={i}, score={score}')

k=2, score=111.68583137120146
k=3, score=89.31584869867916
k=4, score=91.28190595864534
k=5, score=79.69916744628851
k=6, score=90.59868281152153
k=7, score=97.36073809666254
k=8, score=100.79283766488649
k=9, score=97.71744330558076
k=10, score=92.10336304734928
k=11, score=89.05205456831848
k=12, score=91.50217180387268
k=13, score=86.88087304800975
k=14, score=84.25385594771556
k=15, score=84.55916869167594


## 模型部署

### 儲存模型

### 推論預測